In [1]:
from syft.core.adp.data_subject import DataSubject
import syft as sy
import jax
import numpy as np
import functools

from typing import List
from typing import Tuple


kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft
2023-02-08 15:58:31.000136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64:
2023-02-08 15:58:31.014508: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-08 15:58:31.466781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64:
2023-02-08 15:58:31.466922: W tensorflow/stream_executor/plat

In [2]:
domain_client = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into test... done!


In [3]:
alice_data = np.array([1,2,3])
alice_tensor = sy.Tensor(alice_data).annotate_with_dp_metadata(lower_bound=0, upper_bound=10, data_subject="alice")
alice_pointer = alice_tensor.send(domain_client)

Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
perform_api_request_generic <class 'abc.ABCMeta'>


Uploading `97d3fafd624c44fda93de47330b2681d`: 100%|██████████████████| 1/1 [00:00<00:00, 208.07it/s]

perform_api_request_generic <class 'abc.ABCMeta'>


In [5]:
bob_data = np.array([4,5,6])
bob_tensor = sy.Tensor(bob_data).annotate_with_dp_metadata(lower_bound=0, upper_bound=10, data_subject="bob")
bob_pointer = bob_tensor.send(domain_client)

Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
perform_api_request_generic <class 'abc.ABCMeta'>


Uploading `2f7af64cea8b4b728c1b8e486059f1df`: 100%|██████████████████| 1/1 [00:00<00:00, 185.38it/s]

perform_api_request_generic <class 'abc.ABCMeta'>


In [16]:
alice_pointer

PointerId: 97d3fafd624c44fda93de47330b2681d
Status: Ready
Representation: array([0, 8, 3])

(The data printed above is synthetic - it's an imitation of the real data.)

In [6]:
result_pointer = alice_pointer + bob_pointer

In [14]:
res_alice = alice_pointer.publish()

In [15]:
res_alice.get_copy()

[2023-02-08T16:01:45.552723+0200][CRITICAL][logger]][1088928] UnknownPrivateException has been triggered.


UnknownPrivateException: UnknownPrivateException has been triggered.

In [10]:
result_pointer

AttributeError: 'TensorWrappedGammaTensorPointer' object has no attribute 'max_vals'

In [7]:
result = result_pointer.publish()

In [9]:
result.get()

[2023-02-08T16:00:36.625665+0200][CRITICAL][logger]][1088928] UnknownPrivateException has been triggered.


UnknownPrivateException: UnknownPrivateException has been triggered.

In [1]:
class MultiRowPhiTensor(): 
    """
    This class acts like a wrapper over multiple PhiTensors that will offer an helpful API
    instead of having the user to work with multiple PhiTensors of the same structure 
    """
    phi_tensors: np.array[sy.PhiTensor]
    
    def __init__(self, phi_tensors: List[sy.PhiTensor]):
        phi_tensors = np.array(phi_tensors)
        
    def execute_op(self, op, args, kwargs) -> MultiRowPhiTensor:
        return MultiRowPhiTensor(self.phi_tensors.apply(op, args, kwargs))
        
    def reduce_op(self, op, args, kwargs) -> sy.GammaTensor:
        return functools.reduce(op, self.phi_tensors)
    
    def to_flex(self):
        return MultiFlexPhiTensor(self.phi_tensors)
        
        
class MultiFlexPhiTensor():
    """
    This structure is closer to the old PhiTensor, 
    as each value has mapped a DataSubject or DataSubjectArray
    """
    data_subjects_indices: np.array
    data: np.array
    lower_bounds: np.array
    upper_bounds: np.array
    shallow_compute: bool = False
    
    
    def __init__(self, phi_tensors: List[sy.PhiTensor]):
        data_subjects = [phi_tensor.data_subject for phi_tensor in phi_tensors]
        data = np.array([phi_tensor.data for phi_tensor in phi_tensors])
        shape = (len(phi_tensors) *phi_tensors[0].shape)
        lower_bounds = sy.lazyrepeatarray([phi_tensor.lower_bound for phi_tensor in self.phi_tensors], shape=shape)
        upper_bounds = sy.lazyrepeatarray([phi_tensor.upper_bound for phi_tensor in self.phi_tensors], shape=shape)
    
    def __init__(self, data, data_subjects_indices, lower_bounds_array, upper_bounds_array):
        
        if not data.shape == data_subjects_indices.shape == lower_bounds_array.shape == upper_bounds_array.shape:
            raise Exception
        
        self.data = data
        self.data_subjects_indices == data_subjects_indices
        self.lower_bounds = lower_bounds_array
        self.upper_bounds = upper_bounds_array
    
    def ispublic(other):
        return False
    
    def __add__(self, other):
        if ispublic(other):
            data = self.data + other
            lower_bounds = self.lower_bounds + other
            upper_bounds = self.upper_bounds + other
            data_subjects_indices = self.data_subjects_indices
        else: 
            data_subjects_match = (self.data_subjects_indices == other.data_subject_indices).all()
            if data_subjects_match:
                data_subjects_indices = self.data_subjects_indices
                data = self.data + other.data
                lower_bounds = self.lower_bounds + other.lower_bounds
                upper_bounds = self.upper_bounds + other.upper_bounds
            else:
                return self.gamma + other.gamma
        
        return MultiFlexPhiTensor(...)    
    
    
    

NameError: name 'np' is not defined

In [ ]:
# for simple operations:
phi_tensors_obj = MultiRowPhiTensor(...)
phi_tensors_obj *= 2
phi_tensors_obj.sum()

In [ ]:
phi_tensors_obj = MultiFlexPhiTensor(...)
phi_tensors_obj.T 
phi_tensors_obj @ phi_tensors_obj

In [ ]:
class PhiTensor():
    child: comp_array
    bounds: comp_array
    data_subject: sy.DataSubject
    
    def __array_interface__():
        

In [ ]:
x,y,z = PhiTensor(...)
np.array(x,y,z)